In [ ]:
from census_api import DataPull
import polars as pl
dp = DataPull()

In [ ]:
"https://api.census.gov/data/{year}/{dataset}?get=PWSSWGT,PEMLR,PEMARITL&for=state:*&PEEDUCA=39&key=YOUR_KEY_GOES_HERE"
"https://api.census.gov/data/2001/cps/basic/sep?tabulate=weight(PWSSWGT)&col+PEMLR&row+PEMARITL&for=state:*&PEEDUCA=39&key=YOUR_KEY_GOES_HERE"

In [ ]:
url = f"http://api.census.gov/data/{year}/{dataset}?get={variables}for={geo}"
results = requests.get(url).json()

In [ ]:
dp.conn.execute(
        """
    SELECT * FROM sqlite_db.geo_table;
    """
    ).df().head()

In [ ]:
"county:*&in=state:*"

In [ ]:
datased_id = dp.conn.execute(
    """
    SELECT id FROM sqlite_db.dataset_table WHERE dataset='acs-acs5-profile';
    """
).fetchone()[0]
datased_id

In [ ]:
dp.conn.execute(
    """
    SELECT * FROM sqlite_db.geo_interm WHERE dataset_id=?;
    """
, (datased_id,)).pl()

In [ ]:
dp.conn.execute(
    """
    SELECT 
        year_id,
        DISTINCT year_id,
        list(year_id) AS available_ids
    FROM sqlite_db.geo_interm 
        WHERE dataset_id=26 AND geo_id=5;
    """
).list()

In [ ]:
def pull_query(params: list, year: int, geo:str) -> pl.DataFrame:
    # prepare custom census query
    param = ",".join(params)
    base = "https://api.census.gov/data/"
    flow = "/acs/acs5/profile"
    url = f"{base}{year}{flow}?get={param}&for={geo}"
    df = pl.DataFrame(requests.get(url).json())

    # get names from DataFrame
    names = df.select(pl.col("column_0")).transpose()
    names = names.to_dicts().pop()
    names = dict((k, v.lower()) for k, v in names.items())

    # Pivot table
    df = df.drop("column_0").transpose()
    return df.rename(names).with_columns(year=pl.lit(year))

In [ ]:
df = dp.conn.execute(
    """
    SELECT * FROM sqlite_db.dataset_table;
    """
).pl()
df.filter(pl.col("dataset").str.starts_with("acs-acs5-profile"))

In [ ]:
dp.conn.execute(
    """
    SELECT * FROM sqlite_db.year_table;
    """
).df().head()

In [ ]:
dp.conn.execute(
    """
    SELECT * FROM sqlite_db.geo_table;
    """
).df()